<a href="https://colab.research.google.com/github/wizkers007/fooddetection/blob/main/Food_Detection_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold
import tensorflow as tf
from sklearn.metrics import classification_report
from tensorflow.keras.optimizers import Adam
from google.colab import drive
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.preprocessing import image as keras_image
from tensorflow.keras.models import Model
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Mount Google Drive
drive.mount('/content/drive')

# Directory containing images
image_dir = Path('/content/drive/MyDrive/Images3')
filepaths = list(image_dir.glob(r'**/*.jpg'))

# Remove duplicates and ensure valid paths
filepaths = list(set(filepaths))
filepaths = [str(fp) for fp in filepaths if os.path.exists(fp)]

# Extract labels
labels = [os.path.split(os.path.split(fp)[0])[1] for fp in filepaths]

# Convert to DataFrame
images_df = pd.DataFrame({'Filepath': filepaths, 'Label': labels})

# Define KFold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Load MobileNetV2 feature extractor
pretrained_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet', pooling='avg')
feature_extractor = Model(inputs=pretrained_model.input, outputs=pretrained_model.output)

def batch_extract_features(image_paths, batch_size=32):
    """Extract features for images in batches to improve speed."""
    features = []
    num_images = len(image_paths)

    for i in range(0, num_images, batch_size):
        batch_paths = image_paths[i:i+batch_size]
        batch_images = []

        for image_path in batch_paths:
            try:
                img = keras_image.load_img(image_path, target_size=(224, 224))
                img_array = keras_image.img_to_array(img)
                img_array = preprocess_input(np.expand_dims(img_array, axis=0))
                batch_images.append(img_array)
            except Exception as e:
                print(f"Error loading {image_path}: {e}")
                batch_images.append(np.zeros((1, 224, 224, 3)))  # Placeholder

        batch_images = np.vstack(batch_images)
        batch_features = feature_extractor.predict(batch_images, verbose=0)
        features.extend(batch_features)

    return np.array(features)

# Extract features in batches
features = batch_extract_features(filepaths, batch_size=32)

# Normalize features
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

# Encode labels
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

# Number of classes
num_classes = len(np.unique(labels_encoded))

def create_model():
    """Define an improved neural network model."""
    inputs = tf.keras.Input(shape=(1280,))
    x = tf.keras.layers.Dense(256, activation='relu')(inputs)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(0.3)(x)
    x = tf.keras.layers.Dense(128, activation='relu')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(0.3)(x)
    outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(x)

    model = tf.keras.Model(inputs, outputs)
    model.compile(optimizer=Adam(learning_rate=0.0005), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Cross-validation training
fold_no = 1
val_accuracies = []
for train_index, val_index in kf.split(features_scaled):
    print(f"\nTraining fold {fold_no}...")

    # Split the data
    X_train, X_val = features_scaled[train_index], features_scaled[val_index]
    y_train, y_val = labels_encoded[train_index], labels_encoded[val_index]

    # Convert labels to one-hot encoding
    y_train_onehot = tf.keras.utils.to_categorical(y_train, num_classes=num_classes)
    y_val_onehot = tf.keras.utils.to_categorical(y_val, num_classes=num_classes)

    # Create the model
    model = create_model()

    # Define callbacks
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    model_checkpoint = tf.keras.callbacks.ModelCheckpoint(f'model_fold_{fold_no}.h5', save_best_only=True)

    # Train the model
    history = model.fit(
        X_train, y_train_onehot,
        validation_data=(X_val, y_val_onehot),
        epochs=30,
        batch_size=32,
        verbose=1,
        callbacks=[early_stopping, model_checkpoint]
    )

    # Evaluate the model on validation set
    val_loss, val_acc = model.evaluate(X_val, y_val_onehot, verbose=0)
    print(f"Validation Accuracy for fold {fold_no}: {val_acc:.4f}")
    val_accuracies.append(val_acc)

    fold_no += 1

# Print mean validation accuracy
print(f"\nMean Validation Accuracy: {np.mean(val_accuracies):.4f}")


Mounted at /content/drive
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step

Training fold 1...
Epoch 1/30
213/218 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6122 - loss: 1.0682

218/218 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 0.6152 - loss: 1.0597 - val_accuracy: 0.8643 - val_loss: 0.3857
Epoch 2/30
218/218 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.8694 - loss: 0.3548

218/218 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step - accuracy: 0.8694 - loss: 0.3548 - val_accuracy: 0.8971 - val_loss: 0.3121
Epoch 3/30
217/218 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.9091 - loss: 0.2510

218/218 ━━━━━━━━━━━━━━━━━━━━ 6s 18ms/step - accuracy: 0.9091 - loss: 0.2510 - val_accuracy: 0.9040 - val_loss: 0.2871
Epoch 4/30
216/218 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.9415 - loss: 0.1686

218/218 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - accuracy: 0.9414 - loss: 0.1687 - val_accuracy: 0.9155 - val_loss: 0.2628
Epoch 5/30
214/218 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9487 - loss: 0.1425

...

218/218 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.8633 - loss: 0.3780 - val_accuracy: 0.9010 - val_loss: 0.2835
Epoch 3/30
215/218 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9140 - loss: 0.2562

218/218 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.9140 - loss: 0.2561 - val_accuracy: 0.9241 - val_loss: 0.2349
Epoch 4/30
212/218 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9359 - loss: 0.1853

218/218 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.9358 - loss: 0.1855 - val_accuracy: 0.9327 - val_loss: 0.2262
Epoch 5/30
217/218 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9535 - loss: 0.1363

218/218 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.9535 - loss: 0.1363 - val_accuracy: 0.9390 - val_loss: 0.2022
Epoch 6/30
218/218 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 0.9539 - loss: 0.1285 - val_accuracy: 0.9344 - val_loss: 0.2064
Epoch 7/30
218/218 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9696 - loss: 0.0885

218/218 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.9696 - loss: 0.0885 - val_accuracy: 0.9425 - val_loss: 0.1915
Epoch 8/30
218/218 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.9765 - loss: 0.0718 - val_accuracy: 0.9430 - val_loss: 0.1985
Epoch 9/30
218/218 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.9809 - loss: 0.0654 - val_accuracy: 0.9488 - val_loss: 0.1974
Epoch 10/30
218/218 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.9809 - loss: 0.0596 - val_accuracy: 0.9407 - val_loss: 0.2158
Epoch 11/30
218/218 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.9870 - loss: 0.0484 - val_accuracy: 0.9442 - val_loss: 0.2066
Epoch 12/30
218/218 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.9814 - loss: 0.0586 - val_accuracy: 0.9453 - val_loss: 0.2115
Validation Accuracy for fold 5: 0.9425

Mean Validation Accuracy: 0.9410


In [ ]:


# Assuming you have a trained model
# Convert the model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)  # Replace 'model' with your trained model variable
tflite_model = converter.convert()

# Save the converted model to a .tflite file
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)


Saved artifact at '/tmp/tmpoium7hj_'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 1280), dtype=tf.float32, name='keras_tensor_187')
Output Type:
  TensorSpec(shape=(None, 5), dtype=tf.float32, name=None)
Captures:
  136479917107984: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136479917108752: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136479917108560: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136479917106256: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136479917107792: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136479917108176: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136479917109136: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136479917109712: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136479914256016: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136479914250448: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136479917097424: 

In [ ]:
from google.colab import files
files.download('model.tflite')  # This will prompt a download of the model file

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>